In [38]:
import sys
from operator import itemgetter

clonality_list=['denovo_clonal','denovo_subclonal','mother_clonal','mother_subclonal','denovo_clonal_short_indel','denovo_subclonal_short_indel','denovo_?_short_indel']
BP_list = ['.','short_indel','simpleF','complexF','simpleR','complexR','simpleU','complexU','unknown', 'LINE1'] # complexR_1

#info_file = file('00_9_pancreas43.sh')
#output_file = file('summary_pancreas_sv.txt','w')

#info_file = file('00_9_breast21.sh')
#output_file = file('summary_breast_sv.txt','w')

#info_file = file('00_9_invivo_1st.sh')
#output_file = file('summary_invivo_1st_sv.txt','w')

info_file = file('00_9_86_temp.sh')
output_file = file('summary_temp.txt','w')

info_line = info_file.readline().strip()
info_line = info_file.readline().strip()
info_id = []




output_file.write('#sampleID\tDEL\tlarge_DEL\tDUP\tlarge_DUP\tINV\tTRA\tsimpleF\tcomplexF\tsimpleR\tcomplexR\tsimpleU\tcomplexU\tunknown\tforkjumping\tradiation-related\tradiation-subclonal\ttotal\tdose\n')

while info_line:
    info_id.append([info_line.split('\t')[0],info_line.split('\t')[1]])
    info_line = info_file.readline().strip()

target_id = info_id
final_info = []

for i in target_id:
    try:
        input_file = file(i[0] + '.delly.somatic.annotated.filter1.motherinfo.filter2.reviewed.vcf')
    except:
        input_file = file(i[0] + '.delly.somatic.annotated.filter1.reviewed.vcf')
    input_line = input_file.readline().strip()
    input_line = input_file.readline().strip()
    input_del=0;input_dup=0;input_inv=0;input_tra=0;input_sf=0;input_cf=0;input_sr=0;input_cr=0;input_su=0;input_cu=0;input_u=0; sub_sr = 0; input_ld=0; input_ldup=0
    input_cf_cluster = 0; input_cr_cluster =0; input_cu_cluster=0; sub_cr_cluster=0;
    num_fork =0; num_radiation=0; num_total = 0; num_other=0
    while input_line:
        input_split = input_line.split('\t')
        
        input_type = input_split[6]
        input_bp_list_initial = input_split[9]
        input_decision = input_split[11]
        if input_bp_list_initial == 'short_indel':
            input_bp_list = input_bp_list_initial
        else:
            input_bp_list = input_bp_list_initial.split('_')[0]
            
        if input_bp_list in BP_list:
            'blank'
        else:
            print i, 'BP_list'
            print input_line
            
        if input_decision in clonality_list:
            'blank'
        else:
            print i, 'clonality_abn'
            print input_line
            print input_decision
        #if long(input_split[1]) == 138631085 or long(input_split[1]) == 18458599:
        #    print i
        #    print input_line
        
        if input_decision == 'denovo_clonal':
            if input_bp_list == '.':
                if input_type == 'DEL':
                    input_del+=1
                    input_size = long(input_split[8])
                    if input_size > 300000:
                        input_ld+=1
                elif input_type == 'DUP':
                    input_dup+=1
                    input_size = long(input_split[8])
                    if input_size > 100000:
                        input_ldup+=1
                elif input_type == 'INV':
                    input_inv+=1
                    
                    
                elif input_type == 'TRA':
                    input_tra+=1   

                    
                else:
                    print i, 'simple_type'
                    print input_line
            else:
                if input_bp_list == 'simpleF':
                    input_sf+=1

                elif input_bp_list == 'complexF':
                    input_cf+=1
                    input_cf_cluster = max(input_cf_cluster,int(input_bp_list_initial.split('_')[1]))
                elif input_bp_list == 'simpleU':
                    input_su+=1
                    print i[0]
                    print input_line                      
                elif input_bp_list == 'simpleR':
                    input_sr+=1
                   
                elif input_bp_list == 'complexR':
                    input_cf+=1
                    input_cr_cluster = max(input_cr_cluster,int(input_bp_list_initial.split('_')[1]))
                   
                elif input_bp_list == 'complexU':
                    input_cu+=1
                    input_cu_cluster = max(input_cu_cluster,int(input_bp_list_initial.split('_')[1]))
                elif input_bp_list == 'unknown':
                    input_u = max(input_u,int(input_bp_list_initial.split('_')[1]))
                elif input_bp_list == 'LINE1':
                    'blank'
                else:
                    print i, 'complex_type'
                    print input_line
        elif input_decision == 'denovo_subclonal':
            if input_bp_list == '.':
                if input_type == 'INV':
                    num_other+=1
                elif input_type == 'TRA':
                    num_other+=1
            elif input_bp_list == 'complexR':
                sub_cr_cluster = max(sub_cr_cluster,int(input_bp_list_initial.split('_')[1]))
            elif input_bp_list == 'simpleR':
                sub_sr +=1
            else:
                'blank'
            
        input_line = input_file.readline().strip()

    num_fork = input_sf/2 + input_cf_cluster
    num_radiation = input_inv/2 + input_tra/2 + input_sr/2 + input_cr_cluster + num_other / 2 + sub_cr_cluster + sub_sr /2
    num_total = input_ld + input_ldup + input_inv/2 +input_tra/2 + input_sf/2 + input_cf_cluster + input_sr/2 + input_cr_cluster + input_su/2 + input_cu_cluster + input_u + num_other/2 + sub_cr_cluster + sub_sr /2
#    output_file.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % (i[0].split('.')[0], input_del, input_dup, input_inv/2, input_tra/2, input_sf/2, input_cf_cluster, input_sr/2, input_cr_cluster, input_su/2,input_cu_cluster, input_u, num_fork, num_radiation, num_other/2 + sub_cr_cluster, num_total, i[1]) )   
    
    final_info.append([str(i[0].split('.')[0]), str(input_del), str(input_ld) ,str(input_dup),str(input_ldup) , str(input_inv/2), str(input_tra/2), str(input_sf/2), str(input_cf_cluster), str(input_sr/2), str(input_cr_cluster), str(input_su/2), str(input_cu_cluster), str(input_u), str(num_fork), str(num_radiation), str(num_other/2 + sub_cr_cluster + sub_sr/2), str(num_total), str(i[1])])

final_info_sort = sorted(final_info,key=itemgetter(18))

for j in final_info_sort:
    output_file.write('\t'.join(j)+'\n')
    
output_file.close()
print 'THE END'

S1N2P16_4-3
3	14868177	3	74083800	1	5to5	INV	T1	59215623	simpleU_1	0.4	denovo_clonal	25;37;17;13;7;40.48%;31.48%	0;0;34;30	37;42;0;0;0;0.0%;0.0%	.
S1N2P16_4-3
3	74083807	.	.	.	3to.	TRA	.	NA	simpleU_1	.	denovo_clonal	.	.	.	add
B3-0_P19_4Gy-5
5	141830637	13	105322378	1	5to3	TRA	T1	NA	simpleU_1	0.29	denovo_clonal	48;47;19;12;7;28.36%;28.79%	0;0;46;38	119;110;0;0;0;0.0%;0.0%	.
2Gy_2_S04_S
1	154706015	6	56239704	-59	5to3	TRA	T1	NA	simpleU_1	0.39	denovo_clonal	31;65;20;19;7;39.22%;23.53%	0;0;35;35	.	.
2Gy_2_S04_S
1	154706029	6	56239369	.	3to5	TRA	T2	NA	simpleU_1	0.29	denovo_clonal	32;69;13;0;0;28.89%;15.85%	0;0;39;42	.	.
THE END


In [8]:
# use organoid sample SV calls

import os
temp_list = os.listdir("/home/users/jhyouk/06_mm10_SNUH_radiation/33_SV_filter")
file_list = []
for i in temp_list:
    if '.reviewed.vcf' in i:
        file_list.append(i)

input_size = 500
q
#print file_list
output_file = file("temp_TRA.txt",'w')
for i in file_list:
    input_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/33_SV_filter/" + i)
    input_line = input_file.readline().strip()
    input_line = input_file.readline().strip()
    
    input_del = []
    input_pair = []
    
    while input_line:
        if input_line.split('\t')[6] == 'DEL':
            input_del.append(input_line)
        input_line = input_file.readline().strip()
    
    for k in range(len(input_del)):
        input_pair.append([])
    
    input_file.seek(0)
    
    input_line = input_file.readline().strip()
    input_line = input_file.readline().strip()
    while input_line:
        input_split = input_line.split('\t')
        #start from here
        input_chr1 = input_split[0]
        input_pos1 = long(input_split[1])
        input_chr2 = input_split[2]
        input_pos2 = input_split[3]
        
        #input_type = input_split[6]
        #input_gap1 = input_split[45]
        #input_gap2 = input_split[47]
        
        for info_line in input_del:
            info_split = info_line.split('\t')
            info_chr1 = info_split[0]
            info_pos1 = long(info_split[1])
            info_chr2 = info_split[2]
            info_pos2 = long(info_split[3])
            
            if input_line == info_line:
                'blank'
            else:
                if input_pos2 == '.':
                    if input_chr1 == info_chr1:
                        if abs(input_pos1 - info_pos1) < input_size or abs(input_pos1 - info_pos2) < input_size:
                            input_pair[input_del.index(info_line)].append(input_line)

                else:
                    if input_chr1 == info_chr1:
                        if abs(input_pos1 - info_pos1) < input_size or abs(input_pos1 - info_pos2) < input_size:
                            input_pair[input_del.index(info_line)].append(input_line)                       
                    if input_chr2 == info_chr1:
                        if abs(long(input_pos2) - info_pos1) < input_size or abs(long(input_pos2) - info_pos2) < input_size:
                            input_pair[input_del.index(info_line)].append(input_line)
        
        input_line = input_file.readline().strip()
    for l in input_del:
        if len(input_pair[input_del.index(l)]) > 1:
            #print '\n'+i.split('.SV_final')[0]
            output_file.write('##'+i.split('.SV_final')[0] + '\n' + l + '\n')
            for m in input_pair[input_del.index(l)]:
                output_file.write(m+'\n')
        
    input_file.close()
output_file.close()
print 'THE END'

THE END


In [9]:
# choose DEL-TRA combination before this script
# this script is written to consider reads orientation in Lung cancer (not PCAWG vcf files)
input_file = file("temp_TRA.txt")
output_file = file("temp_TRA_ori.txt",'w')
input_size = 500

input_line = input_file.readline().strip()

input_tra = []
input_mate = []
while input_line:
    if input_line[0:2] == '##':
        if len(input_tra) == 2:
            if input_mate[0][0] == input_mate[1][0] and abs(input_mate[0][1] - input_mate[1][1]) < input_size:                
                output_file.write(info_1 + '\n')
                output_file.write(info_2 + '\n')
                for i in input_tra:
                    output_file.write(i+'\n')
           
        input_tra = []
        input_mate = []
        info_1 = input_line
        input_line = input_file.readline().strip()
        info_2 = input_line
        
        info_split = input_line.split('\t')
        info_chr1 = info_split[0]
        info_pos1 = long(info_split[1])
        info_chr2 = info_split[2]
        info_pos2 = long(info_split[3])

    
    else:        
        input_split = input_line.split('\t')
        input_chr1 = input_split[0]
        input_pos1 = long(input_split[1])
        input_chr2 = input_split[2]
        input_pos2 = input_split[3]
        input_ori1 = input_split[5].split('to')[0]
        input_ori2 = input_split[5].split('to')[1]
        input_type = input_split[6]
        #print input_split
        #break
        if input_type != 'TRA':
            'blank'
        else:
            if input_pos2 == '.':
                if input_chr1 == info_chr1 and abs(input_pos1 - info_pos1) < input_size and input_ori1 == '5' :
                    input_tra.append(input_line)
                elif input_chr1 == info_chr2 and abs(input_pos1 - info_pos2) < input_size and input_ori1 == '3' :
                    input_tra.append(input_line)
                else:
                    'blank'
            else:
                input_pos2 = long(input_pos2)
                if input_chr1 == info_chr1 and abs(input_pos1 - info_pos1) < input_size and input_ori1 == '5' :
                    input_tra.append(input_line)
                    input_mate.append([input_chr2,input_pos2])
                elif input_chr2 == info_chr1 and abs(input_pos2 - info_pos1) < input_size and input_ori2 == '5' :
                    input_tra.append(input_line)
                    input_mate.append([input_chr1,input_pos1])
                elif input_chr1 == info_chr2 and abs(input_pos1 - info_pos2) < input_size and input_ori1 == '3' :
                    input_tra.append(input_line)
                    input_mate.append([input_chr2,input_pos2])
                elif input_chr2 == info_chr2 and abs(input_pos2 - info_pos2) < input_size and input_ori2 == '3' :
                    input_tra.append(input_line)
                    input_mate.append([input_chr1,input_pos1])
                else:
                    'blank'
            
    input_line = input_file.readline().strip()
if len(input_tra) == 2:
    #print input_mate
    #print abs(input_mate[0][1] - input_mate[1][1])
    if input_mate[0][0] == input_mate[1][0] and abs(input_mate[0][1] - input_mate[1][1]) < input_size:                
        output_file.write(info_1 + '\n')
        output_file.write(info_2 + '\n')
        for i in input_tra:
            output_file.write(i+'\n')

output_file.close()
print 'THE END'

THE END


In [19]:
#check balanced breakpoints?

from operator import itemgetter

input_file = file("S1N2P15_2-6.delly.somatic.annotated.filter1.motherinfo.filter2.reviewed.vcf")
input_line = input_file.readline().strip()
input_line = input_file.readline().strip()
n_total = 0

list_dic = []
while input_line:
    input_split = input_line.split('\t')
    if input_split[9] == 'complexF_2' or input_split[9] == 'complexF_3':
        n_total+=1
        input_chr1 = int(input_split[0].replace('X','20'))
        input_pos1 = long(input_split[1])
        list_dic.append([input_chr1,input_pos1,input_split[5].split('to')[0]])
        
        input_chr2 = int(input_split[2].replace('X','20'))
        input_pos2 = long(input_split[3])
        list_dic.append([input_chr2,input_pos2,input_split[5].split('to')[1]])
        
    input_line = input_file.readline().strip()
    
print n_total

list_dic.sort(key=itemgetter(0,1))
for i in list_dic:
    print i
    
    
    


29
[12, 71952831L, '5']
[12, 72460626L, '3']
[12, 78355092L, '5']
[12, 92742630L, '3']
[12, 104509376L, '5']
[12, 104527168L, '5']
[12, 104527401L, '3']
[12, 104533963L, '3']
[12, 105982247L, '5']
[12, 113166148L, '5']
[12, 113272699L, '5']
[12, 113276994L, '3']
[12, 115300227L, '5']
[12, 115306465L, '3']
[12, 120002791L, '5']
[20, 49835332L, '3']
[20, 49835354L, '5']
[20, 63247803L, '3']
[20, 63247813L, '5']
[20, 83035320L, '5']
[20, 83035514L, '5']
[20, 83035516L, '3']
[20, 83665715L, '5']
[20, 83666830L, '3']
[20, 83812752L, '5']
[20, 83814401L, '5']
[20, 83818267L, '5']
[20, 83818276L, '3']
[20, 83818523L, '3']
[20, 83851224L, '5']
[20, 83984585L, '3']
[20, 107456322L, '5']
[20, 107461730L, '5']
[20, 107463175L, '3']
[20, 107463361L, '3']
[20, 107463999L, '3']
[20, 107479770L, '5']
[20, 107484181L, '3']
[20, 107495539L, '3']
[20, 109298726L, '5']
[20, 111078600L, '5']
[20, 111080213L, '3']
[20, 111081524L, '3']
[20, 114319882L, '5']
[20, 119367360L, '3']
[20, 119367366L, '5']
[20, 

In [37]:
#short indel number check
import os
import numpy as np
temp_list = os.listdir("/home/users/jhyouk/06_mm10_SNUH_radiation/33_SV_filter")
file_list = []
for i in temp_list:
    if '.reviewed.vcf' in i:
        file_list.append(i)

info_file = file("00_9_86_temp.sh")
info_line = info_file.readline().strip()
info_line = info_file.readline().strip()
dic_info = {}
while info_line:
    dic_info[info_line.split('\t')[0]] = info_line.split('\t')[1]
    info_line = info_file.readline().strip()
    
stat_info = ['0','0.3','1','2','4','8','20']
stat_result = [[],[],[],[],[],[],[]]


for i in file_list:
    j=0
    input_file = file("/home/users/jhyouk/06_mm10_SNUH_radiation/33_SV_filter/" + i)
    input_line = input_file.readline().strip()
    input_line = input_file.readline().strip()
    while input_line:
        input_split = input_line.split('\t')
        try:
            input_split[11]
        except:
            print i
        if input_split[11] == 'denovo_clonal_short_indel':
            j+=1
        
        input_line = input_file.readline().strip()
    try:
        stat_result[stat_info.index(dic_info[i.split('.delly')[0]])].append(j)
    except:
        print i.split('.delly')[0]
for k in stat_result:
    print np.mean(k)
    


20Gy_1_S03_S
Liver_2Gy2_SO4
C3SO3
Panc_20Gy1_SO2
0.5263157894736842
0.75
0.8461538461538461
1.4615384615384615
1.5384615384615385
1.2857142857142858
2.75
